# Document scanning tool

## Setup

### Setup env

In [ ]:
# create env
!conda create -n search-3gpp python==3.11
# activate env
!conda activate search-3gpp
# install requirements
!pip install -r requirements.txt

### Install OS libraries

In [ ]:
# For antiword
!sudo apt-get install antiword

# For Elasticsearch (assuming Debian/Ubuntu)
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.0-amd64.deb
!sudo apt-get update && sudo dpkg -i elasticsearch-7.10.0-amd64.deb

### Imports

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import argparse
from elasticsearch import Elasticsearch
from dl_docs import *
from process_docs import *
from inverse_index import *

%load_ext autoreload
%autoreload 2

## Index & search

### Download docs from 3gpp

In [2]:
url = 'https://www.3gpp.org/ftp/Specs/2023-06'  # URL of the webpage to be parsed

# Download zip files
download_docs_from_3gpp(url, save_dir='../data/downloaded_docs', max_files=100)
# or from terminal
# !python dl_docs.py --url 'https://www.3gpp.org/ftp/Specs/2023-06' --save_dir '../data/downloaded_docs' --max_files 100

  0%|          | 0/12 [00:02<?, ?it/s]


Max file limit reached. Returning 100 files.


100%|██████████| 100/100 [02:39<00:00,  1.60s/it]

Done!


### Process docs

In [2]:
# extract text
doc2txt(src_dir='../data/downloaded_docs', dst_dir='../data/raw_docs')
# or from terminal
# !python process_docs.py --src_dir ../data/downloaded_docs --dst_dir ../data/raw_docs

sh: 1: Syntax error: "(" unexpected


### Create inverse index

In [ ]:
# Start Elasticsearch
!sudo service elasticsearch start

In [2]:
# create elasticsearch object
es = Elasticsearch("http://localhost:9200")

# Index the .txt files
index_files_to_es(es, idx_name='3gpp_docs', docs_dir='../data/raw_docs')

# or from terminal
# !python inverse_index.py --idx_name 3gpp_docs --docs_dir ../data/raw_docs

Indexed files to Elasticsearch.


### Run flask app for searching index

In [ ]:
!python app.py --doc_dir ../data/raw_docs